In [1]:
import pandas as pd
import numpy as np

# Feature Selection

### Load in the data

In [2]:
data = pd.read_csv('dataset/spotify_songs_cleaned.csv')
data.head()

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_genre,danceability,energy,key,loudness,speechiness,liveness,valence,tempo,duration_ms,popularity_category,tempo_category,loudness_category,release_month
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,pop,0.748,0.916,6,-2.634,0.0583,0.0653,0.518,122.036,194754,High Popularity,Fast,Loud,June
1,Memories - Dillon Francis Remix,Maroon 5,67,Memories (Dillon Francis Remix),2019-12-13,pop,0.726,0.815,11,-4.969,0.0373,0.3570,0.693,99.972,162600,High Popularity,Moderate,Loud,December
2,All the Time - Don Diablo Remix,Zara Larsson,70,All the Time (Don Diablo Remix),2019-07-05,pop,0.675,0.931,1,-3.432,0.0742,0.1100,0.613,124.008,176616,High Popularity,Fast,Loud,July
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,Call You Mine - The Remixes,2019-07-19,pop,0.718,0.930,7,-3.778,0.1020,0.2040,0.277,121.956,169093,Medium Popularity,Fast,Loud,July
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,Someone You Loved (Future Humans Remix),2019-03-05,pop,0.650,0.833,1,-4.672,0.0359,0.0833,0.725,123.976,189052,High Popularity,Fast,Loud,March
